<a href="https://colab.research.google.com/github/athishsreeram/Text-to-Video/blob/main/Text%2BImg_to_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pillow gTTS moviepy pydub

In [2]:
from PIL import ImageFont, ImageDraw, Image
from gtts import gTTS
from moviepy.editor import ImageSequenceClip, AudioFileClip
import argparse
from pydub import AudioSegment
import numpy as np
import os

# Variables for customization
TEXT_SPEED = 24  # frames per second
TEXT_COLOR = (0, 0, 0)
FONT_PATH = "/content/DMSerifDisplay-Regular.ttf"  # Path to .ttf font file (change this to your font file)
FONT_SIZE = 180
TIMING_ADJUSTMENT = -0.3  # Adjusts the duration of each word in the video
BACKGROUND_IMAGE_PATH = "/content/1.png"  # Path to the background image file
VIDEO_SIZE = (1080, 1920)

def get_word_durations(textfile, audiofile):
    with open(textfile, "r") as f:
        lines = f.read()

    words = lines.split()

    # Load the audio file
    audio = AudioSegment.from_file(audiofile)
    full_audio_duration = len(audio) / 1000  # duration in seconds

    # Calculate average duration per word
    avg_word_duration = full_audio_duration / len(words)

    # Print duration for each word
    for i, word in enumerate(words):
        print(f"Word: '{word}' Duration: {avg_word_duration:.2f} seconds")


def text_to_video(textfile, outputfile):
    with open(textfile, "r") as f:
        lines = f.read()

    words = lines.split()
    images = []
    durations = []

    fnt = ImageFont.truetype(FONT_PATH, FONT_SIZE)

    # Generate speech for the whole text and save as a temporary file
    tts = gTTS(text=lines, lang="en")
    tts.save("temp.mp3")

    # Measure the speech duration using pydub
    full_audio = AudioSegment.from_file("temp.mp3")
    full_audio_duration = len(full_audio) / 1000  # duration in seconds
    avg_word_duration = full_audio_duration / len(words)  # average duration per word
    durations.append(avg_word_duration + TIMING_ADJUSTMENT)  # Adjust frame duration based on average word duration and timing adjustment

    # Load the background image
    bg_img = Image.open(BACKGROUND_IMAGE_PATH).resize(VIDEO_SIZE)

    for i, word in enumerate(words):
        # Calculate text size and position
        text_width, text_height = fnt.getsize(word)
        position = ((VIDEO_SIZE[0] - text_width) / 2, (VIDEO_SIZE[1] - text_height) / 2)

        # Create a new image with the background
        img = bg_img.copy()
        d = ImageDraw.Draw(img)
        d.text(position, word, font=fnt, fill=TEXT_COLOR)

        images.append(np.array(img))
        durations.append(avg_word_duration)  # Set frame duration based on average word duration

    audioclip = AudioFileClip("temp.mp3")
    clip = ImageSequenceClip(images, durations=durations)
    clip = clip.set_audio(audioclip)

    clip.fps = TEXT_SPEED
    clip.write_videofile(outputfile, codec="libx264")



if __name__ == "__main__":
    text_to_video("N.txt", "output.mp4")
    get_word_durations("N.txt", "temp.mp3")
    # Remove the temporary file
    # os.remove("temp.mp3")


  text_width, text_height = fnt.getsize(word)



Moviepy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4
